
# Session 1 and 2 (Part A) — Agentic AI: Labs 

This notebook contains **reference implementations** for Session 1:
- Lab 1: Reactive Chatbot (baseline)
- Lab 2: Tool-Using Agent
- Lab 3: Planning Agent
- Lab 4: Memory Demo
- Lab 5: Feedback & Retry

> ⚠️ **Note**: Internet access may be disabled. All examples include **mocked** functions for smooth execution, plus commented code showing how to switch to real APIs later.



## Setup

We define a tiny **mock LLM** and **mock tools** so everything runs offline.
If you want to use real APIs later:
- Replace `mock_chat()` with an OpenAI/Anthropic client call.
- Replace `mock_get_weather()` with a real HTTP request (e.g., `wttr.in` or a weather API).
- Replace `mock_search_wikipedia()` with a real Wikipedia client.


In [1]:

from dataclasses import dataclass
from typing import List, Dict, Any
import random
import time

# ------------------ Mock LLM ------------------
def mock_chat(messages: List[Dict[str, str]], model: str = "mock-gpt") -> str:
    """A very small mock that returns deterministic but helpful text."""
    last_user = next((m["content"] for m in reversed(messages) if m["role"] == "user"), "")
    if "break down" in last_user.lower() or "steps" in last_user.lower():
        return ("1) Clarify requirements\n2) Plan tasks\n3) Execute tools\n"
                "4) Verify outcomes\n5) Summarize & next steps")
    if "capital of france" in last_user.lower():
        return "Paris."
    if "dinner" in last_user.lower():
        return "Consider a casual bistro in River North and a lakefront walk afterwards."
    return f"(mocked) Response to: {last_user[:80]}..."

# ------------------ Mock Tools ------------------
def mock_get_weather(city: str) -> str:
    temperatures = {"chicago": "+22°C 🌤️", "new york": "+24°C 🌥️", "london": "+17°C 🌧️"}
    return f"{city.title()}: {temperatures.get(city.lower(), '+20°C ☁️ (mock)')}"

def mock_search_wikipedia(query: str) -> str:
    return f"(mock) Top summary for '{query}': This is a concise encyclopedia-style overview."

# Retry helper
def with_retry(fn, retries=3, delay=0.5):
    for i in range(retries):
        try:
            return fn()
        except Exception as e:
            if i == retries - 1:
                raise
            time.sleep(delay)

print("✅ Setup complete (mock LLM & tools ready).")


✅ Setup complete (mock LLM & tools ready).



## Lab 1 — Reactive Chatbot (Baseline)

A minimal, **reactive** chatbot. No tools. No memory. No autonomy.


In [3]:

def chatbot(question: str) -> str:
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": question},
    ]
    return mock_chat(messages)

print("Q: What is the capital of France?")
print("A:", chatbot("What is the capital of France?"))


Q: What is the capital of France?
A: Paris.



## Lab 2 — Tool-Using Agent

A tiny **agent** that decides whether to call a **weather tool** or fallback to the LLM.


In [5]:

def tool_using_agent(user_input: str) -> str:
    text = user_input.lower()
    if "weather" in text:
        # naive city extraction
        city = "Chicago"
        for name in ["Chicago", "New York", "London"]:
            if name.lower() in text:
                city = name
                break
        print("Agent: I decided to call the weather tool.")
        weather = mock_get_weather(city)
        return f"Weather result → {weather}"
    else:
        print("Agent: No tool needed; using LLM.")
        return mock_chat([{"role": "user", "content": user_input}])

print(tool_using_agent("What's the weather in Chicago today?"))
print(tool_using_agent("Suggest an evening dinner plan."))


Agent: I decided to call the weather tool.
Weather result → Chicago: +22°C 🌤️
Agent: No tool needed; using LLM.
Consider a casual bistro in River North and a lakefront walk afterwards.


In [11]:
import requests 
from openai import OpenAI

#client = OpenAI()


def get_weather(city: str) -> str:
    url = f"http://wttr.in/{city}?format=3"
    r = requests.get(url, timeout=8)
    r.raise_for_status()
    return r.text 


def tool_using_agent(user_input: str) -> str:
    text = user_input.lower()
    if "weather" in text:
        # naive city extraction
        city = "Chicago"
        for name in ["Chicago", "New York", "London"]:
            if name.lower() in text:
                city = name
                break
        try:
            weather = get_weather(city)
            return f"I checked the weather tool: {weather}"
        except Exception as e:
            return f"Sorry weather tool failed: {e}"
    else:
        print("Agent: No tool needed; using LLM.")
        return mock_chat([{"role": "user", "content": user_input}])

print(tool_using_agent("What's the weather in London today?"))
print(tool_using_agent("Suggest an evening dinner plan."))


I checked the weather tool: London: ⛅️  +54°F

Agent: No tool needed; using LLM.
Consider a casual bistro in River North and a lakefront walk afterwards.



## Lab 3 — Planning Agent

Ask the LLM to **break down a goal** into steps.


In [14]:

def planning_agent(goal: str) -> str:
    messages = [
        {"role": "system", "content": "You are a planning agent."},
        {"role": "user", "content": f"Break down the steps to: {goal}"},
    ]
    return mock_chat(messages)

print(planning_agent("Organize a team offsite event"))


1) Clarify requirements
2) Plan tasks
3) Execute tools
4) Verify outcomes
5) Summarize & next steps



## Lab 4 — Memory Demo

Keep simple **short-term memory** across calls.


In [16]:

conversation_memory = []

def memory_agent(user_input: str) -> str:
    conversation_memory.append(user_input)
    return f"Received: {user_input} | Memory: {conversation_memory[-3:]}"

print(memory_agent("Book flight to New York"))
print(memory_agent("Change it to tomorrow"))
print(memory_agent("Add one extra bag"))


Received: Book flight to New York | Memory: ['Book flight to New York']
Received: Change it to tomorrow | Memory: ['Book flight to New York', 'Change it to tomorrow']
Received: Add one extra bag | Memory: ['Book flight to New York', 'Change it to tomorrow', 'Add one extra bag']


In [18]:
conversation_memory

['Book flight to New York', 'Change it to tomorrow', 'Add one extra bag']


## Lab 5 — Feedback & Retry

Retry an action that may fail. Here we **simulate** failure with randomness.


In [24]:

def flaky_action():
    if random.random() < 0.6:
        raise RuntimeError("Transient tool error.")
    return "Action succeeded ✅"

def feedback_agent():
    attempts = 0
    max_attempts = 3
    while attempts < max_attempts:
        try:
            result = flaky_action()
            return f"{result} (attempt {attempts+1})"
        except Exception as e:
            attempts += 1
            print(f"Attempt {attempts} failed: {e}. Retrying...")
    return "Failed after 3 attempts. Please try again later."

print(feedback_agent())


Action succeeded ✅ (attempt 1)


In [26]:

def trip_planner(city: str = "Chicago"):
    weather = mock_get_weather(city)
    plan = mock_chat([
        {"role": "system", "content": "You are a trip planning agent."},
        {"role": "user", "content": f"The weather is: {weather}. Suggest an evening dinner plan."}
    ])
    return f"Weather: {weather}\nPlan: {plan}"

print(trip_planner("Chicago"))


Weather: Chicago: +22°C 🌤️
Plan: Consider a casual bistro in River North and a lakefront walk afterwards.


In [28]:

class ToolAgentWithMemory:
    def __init__(self, memory_size: int = 3):
        self.memory = []
        self.memory_size = memory_size

    def remember(self, q: str):
        self.memory.append(q)
        if len(self.memory) > self.memory_size:
            self.memory.pop(0)

    def handle(self, query: str) -> str:
        self.remember(query)
        prefix = f"(memory: {self.memory})\n"
        return prefix + select_tool_and_answer(query)

agent = ToolAgentWithMemory()
print(agent.handle("What's the weather in Chicago?"))
print(agent.handle("Search Wikipedia about Large Language Models"))
print(agent.handle("Recommend a dinner plan"))
print(agent.handle("What's the weather in New York?"))


NameError: name 'select_tool_and_answer' is not defined


## Take-Home Assignment 2 — Mini Planner Agent (Execute One Step)

- Break down a goal into steps.
- **Execute one step** (e.g., fetch weather).


In [ ]:

def execute_step(step: str) -> str:
    step_l = step.lower()
    if "weather" in step_l:
        return mock_get_weather("Chicago")
    return f"(mock) Executed step: {step}"

def mini_planner_with_execution(goal: str):
    steps_text = planning_agent(goal)
    steps = [s.strip() for s in steps_text.split("\n") if s.strip()]
    report = []
    for i, step in enumerate(steps, 1):
        result = execute_step(step)
        report.append(f"Step {i}: {step} -> {result}")
    return "\n".join(report)

print(mini_planner_with_execution("Plan a day in Chicago"))



## Take-Home Assignment 3 — Feedback & Retry (Decorator)

Wrap any function with a retry **decorator**.


In [ ]:

def retry(retries=3, delay=0.3):
    def decorator(fn):
        def wrapper(*args, **kwargs):
            for i in range(retries):
                try:
                    return fn(*args, **kwargs)
                except Exception as e:
                    if i == retries - 1:
                        return f"Failed after {retries} attempts: {e}"
                    time.sleep(delay)
            return None
        return wrapper
    return decorator

@retry(retries=3, delay=0.2)
def sometimes_fails():
    if random.random() < 0.7:
        raise RuntimeError("Network hiccup")
    return "Success ✅"

print(sometimes_fails())



---

## Switching to Real APIs (Optional)

**OpenAI (pseudo-code):**
```python
from openai import OpenAI
client = OpenAI()
resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello"}]
)
print(resp.choices[0].message["content"])
```

**Weather via wttr.in:**
```python
import requests
print(requests.get("http://wttr.in/Chicago?format=3", timeout=10).text)
```


In [ ]:
import requests 
from openai import OpenAI

client = OpenAI()

def tool_using_agent(user_input: str) -> str:
    text = user_input.lower()
    if "weather" in text:
        # naive city extraction
        city = "Chicago"
        for name in ["Chicago", "New York", "London"]:
            if name.lower() in text:
                city = name
                break
        print("Agent: I decided to call the weather tool.")
        weather = mock_get_weather(city)
        return f"Weather result → {weather}"
    else:
        print("Agent: No tool needed; using LLM.")
        return mock_chat([{"role": "user", "content": user_input}])

print(tool_using_agent("What's the weather in Chicago today?"))
print(tool_using_agent("Suggest an evening dinner plan."))
